In [1]:
from __future__ import division
def feature_eng(y, fs, vis=False):
    
    import librosa
    import librosa.display
    import numpy as np
    import maad
    from soundsig.sound import BioSound
    import matplotlib.pyplot as plt
    import pandas as pd
    import warnings
    warnings.simplefilter('ignore')
    
    # spectrum
    ps = np.abs(np.fft.fft(y))**2
    time_step = 1/fs
    freqs = np.fft.fftfreq(y.size, time_step)
    ps = ps[0:int((len(ps)/2)-1)] # take out the negative freq
    freqs = freqs[0:int((len(freqs)/2)-1)]

    # melspectrogram
    S = librosa.feature.melspectrogram(y=y, sr=fs)
    S_dB = librosa.power_to_db(S, ref=np.max)
    
    # alpha indices
    Sxx_power,tn,fn,ext = maad.sound.spectrogram (y, fs, mode='psd')
    df_temporal_indices = maad.features.all_temporal_alpha_indices(y, fs)
    df_spectral_indices, _ = maad.features.all_spectral_alpha_indices(Sxx_power,tn,fn, extent=ext)
    df_indices = pd.concat([df_temporal_indices,df_spectral_indices], axis=1)
    
    # soundsig for modulation power spectrum
    
    myBioSound = BioSound(soundWave=y, fs=fs)
    myBioSound.mpsCalc(window=0.1, Norm = True)
    
    # reduce the dimension of MPS to one quadrant
    len1 = int(len(myBioSound.wf-1)/2)
    len2 = int(len(myBioSound.wt-1)/2)
    quad1 = myBioSound.mps[len1:,len2:]
    quad2 = np.fliplr(myBioSound.mps[len1:,:len2+1])
    mps = (quad1+quad2)/2
    wf = myBioSound.wf[len1:]
    wt = myBioSound.wt[len2:]
    
    if vis==True:
        plt.figure()
        plt.plot(np.arange(0,y.size/fs,1/fs), y)
        plt.show()
        
        fig, ax = plt.subplots()
        img = librosa.display.specshow(S_dB, x_axis='time',
                                 y_axis='mel', sr=fs,
                                 fmax=fs/2, ax=ax)
        fig.colorbar(img, ax=ax, format='%+2.0f dB')
        ax.set(title='Mel-frequency spectrogram')
        
        DBNOISE=50
        plt.figure()
        plt.clf()
        cmap = plt.get_cmap('jet')
        ex = (myBioSound.wt.min(), myBioSound.wt.max(), myBioSound.wf.min()*1e3, myBioSound.wf.max()*1e3)
        logMPS = 10.0*np.log10(myBioSound.mps)
        maxMPS = logMPS.max()
        minMPS = maxMPS-DBNOISE
        logMPS[logMPS < minMPS] = minMPS
        plt.imshow(logMPS, interpolation='nearest', aspect='auto', origin='lower', cmap=cmap, extent=ex)
        plt.ylabel('Spectral Frequency (Cycles/KHz)')
        plt.xlabel('Temporal Frequency (Hz)')
        plt.colorbar()
        plt.ylim((0,myBioSound.wf.max()*1e3))
        plt.title('Modulation Power Spectrum')
        plt.show()
        
    
    return ps, S_dB, df_indices, mps, wt, wf
    

In [2]:
def preproc(file, fs=16000, vis=False):
    import librosa
    import numpy as np
    import matplotlib.pyplot as plt
    import noisereduce as nr
    
    raw_y, fs = librosa.load(file, sr=fs, duration=10, mono = True)
    y_mono_rs = raw_y - np.mean(raw_y) # remove DC
    rms = np.sqrt(np.mean(y_mono_rs**2)) # get rms
    y = y_mono_rs/(rms/0.1) # normalize the rms to 0.1
    
    fg_y = nr.reduce_noise(y=y, sr=fs)
    bg_y = y - fg_y
    
    
    if vis == True: print('++++++++++++++++++++++++ raw ++++++++++++++++++++++++')
    ps, S_dB, df_indices, mps, wt, wf = feature_eng(y, fs, vis)
    if vis == True: print('++++++++++++++++++++ foreground ++++++++++++++++++++')
    ps_fg, S_dB_fg, df_indices_fg, mps_fg, wt_fg, wf_fg = feature_eng(fg_y, fs, vis)
    if vis == True: print('++++++++++++++++++++ background ++++++++++++++++++++')
    ps_bg, S_dB_bg, df_indices_bg, mps_bg, wt_bg, wf_bg = feature_eng(bg_y, fs, vis)
    
    output = {'y': y, 'fg_y': fg_y, 'bg_y': bg_y, 'fs': fs,
             'ps': ps, 'S_dB': S_dB, 'df_indices': df_indices, 'mps': mps, 'wt': wt, 'wf': wf, 
             'ps_fg': ps_fg, 'S_dB_fg': S_dB_fg, 'df_indices_fg': df_indices_fg, 'mps_fg': mps_fg, 'wt_fg': wt_fg, 'wf_fg': wf_fg,
             'ps_bg': ps_bg, 'S_dB_bg': S_dB_bg, 'df_indices_bg': df_indices_bg, 'mps_bg': mps_bg, 'wt_bg': wt_bg, 'wf_bg': wf_bg}
    
    return output
    

In [3]:
# scan data directories
import glob
import pandas as pd

nature_file_list = []
nature_file_list += glob.glob('../data/raw/AmbisonicSoundLibrary/nature/*')
nature_file_list += glob.glob('../data/raw/GoogleAudioSet/Outside, rural or natural/*')
nature_file_list += glob.glob('../data/raw/youtube/NatureSoundscapes/*')
nature_file_list += glob.glob('../data/raw/youtube/NomadicAmbience_nature/*')
nature_file_list += glob.glob('../data/raw/S2L_LULC/non_urban/*')
nature_file_list += glob.glob('../data/raw/S2L_LULC/urban_0_25/*')

city_file_list = []
city_file_list += glob.glob('../data/raw/GoogleAudioSet/Outside, urban or manmade/*')
city_file_list += glob.glob('../data/raw/youtube/NomadicAmbience_city/*')
city_file_list += glob.glob('../data/raw/S2L_LULC/urban_26_100/*')

In [4]:
# as there are too many files in SONYC, so only a few are sampled
from random import seed, sample
seed(23)
SONYC_file_list = sample(glob.glob('../data/raw/SONYC/**/*.wav', recursive=True),800)
city_file_list += SONYC_file_list

pd.DataFrame({'nature_file_list': nature_file_list}).to_csv('../data/raw/nature_file_list.csv')
pd.DataFrame({'city_file_list': city_file_list}).to_csv('../data/raw/city_file_list.csv')

In [5]:
# testing cell
# preproc(city_file_list[0])

In [6]:
import time
import pickle
import os

from joblib import Parallel, delayed

def run_preproc(file_name):
    save_file_name = '../data/interim/'+file_name[12:-4]+'.pkl'
    if not os.path.isfile(save_file_name): # run the script only if the file does not exist
        if not os.path.exists(save_file_name.rsplit('/', 1)[0]): # create the folder if the folder does not exist
            os.makedirs(save_file_name.rsplit('/', 1)[0]) # extract the folder of the file path
            

        start_time = time.time()

        output = preproc(file_name, vis = False)
        f = open(save_file_name,'wb') # create a binary pickle file 
        pickle.dump(output,f)
        f.close()

        print("--- %s seconds ---" % (time.time() - start_time))
        

# run the process in parallel
Parallel(n_jobs=3)(delayed(run_preproc)(file_name) for file_name in nature_file_list+city_file_list)

--- 10.39131498336792 seconds ---
--- 6.2836689949035645 seconds ---
--- 6.308449029922485 seconds ---
--- 6.254638910293579 seconds ---
--- 6.074193954467773 seconds ---
--- 6.39683985710144 seconds ---
--- 5.999579191207886 seconds ---
--- 6.191865921020508 seconds ---
--- 8.889364957809448 seconds ---
--- 9.506667137145996 seconds ---
--- 6.826747894287109 seconds ---
--- 6.291080951690674 seconds ---
--- 6.7837841510772705 seconds ---
--- 6.841970920562744 seconds ---
--- 7.040736198425293 seconds ---
--- 6.524389982223511 seconds ---
--- 6.644038915634155 seconds ---
--- 6.84984827041626 seconds ---
--- 7.079504013061523 seconds ---
--- 5.794532775878906 seconds ---
--- 6.128819942474365 seconds ---
--- 6.07197117805481 seconds ---
--- 6.501261949539185 seconds ---
--- 6.374599933624268 seconds ---
--- 6.141749858856201 seconds ---
--- 6.1239402294158936 seconds ---
--- 6.213067054748535 seconds ---
--- 6.632147312164307 seconds ---
--- 6.48709511756897 seconds ---
--- 6.310248136

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


--- 10.447558164596558 seconds ---
--- 6.253890037536621 seconds ---
--- 6.4689929485321045 seconds ---
--- 6.125679969787598 seconds ---
--- 5.614037036895752 seconds ---
--- 6.154653072357178 seconds ---
--- 6.067126035690308 seconds ---
--- 6.034637928009033 seconds ---
--- 7.9901511669158936 seconds ---
--- 9.87023377418518 seconds ---
--- 6.785912036895752 seconds ---
--- 6.30908727645874 seconds ---
--- 6.546128988265991 seconds ---
--- 7.1600120067596436 seconds ---
--- 6.989638090133667 seconds ---
--- 6.506600856781006 seconds ---
--- 6.504542112350464 seconds ---
--- 6.799320697784424 seconds ---
--- 7.034280061721802 seconds ---
--- 6.81331992149353 seconds ---
--- 6.072148084640503 seconds ---
--- 6.151094913482666 seconds ---
--- 6.215941905975342 seconds ---
--- 6.622235059738159 seconds ---
--- 6.0295090675354 seconds ---
--- 6.201658248901367 seconds ---
--- 6.374199151992798 seconds ---
--- 6.679703712463379 seconds ---
--- 6.592844009399414 seconds ---
--- 6.367399930

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


--- 8.799047946929932 seconds ---
--- 5.332535266876221 seconds ---
--- 6.073585033416748 seconds ---
--- 5.533267974853516 seconds ---
--- 5.156569004058838 seconds ---
--- 4.9960198402404785 seconds ---
--- 4.981958866119385 seconds ---
--- 4.95278000831604 seconds ---
--- 5.182962894439697 seconds ---
--- 5.680354833602905 seconds ---
--- 5.1174962520599365 seconds ---
--- 5.155012130737305 seconds ---
--- 5.256070852279663 seconds ---
--- 5.3038551807403564 seconds ---
--- 5.245671033859253 seconds ---
--- 5.017155647277832 seconds ---
--- 5.077536106109619 seconds ---
--- 5.569187879562378 seconds ---
--- 5.086421012878418 seconds ---
--- 5.632535696029663 seconds ---
--- 5.13420295715332 seconds ---
--- 5.034749984741211 seconds ---
--- 5.189101934432983 seconds ---
--- 5.194904804229736 seconds ---
--- 5.317945241928101 seconds ---
--- 5.513818740844727 seconds ---
--- 5.695349931716919 seconds ---
--- 5.269685983657837 seconds ---
--- 5.3131263256073 seconds ---
--- 5.217097997

...

--- 14.655373096466064 seconds ---
--- 8.731701135635376 seconds ---
--- 8.957275152206421 seconds ---
--- 9.000128030776978 seconds ---
--- 9.130801916122437 seconds ---
--- 8.37956190109253 seconds ---
--- 9.315713167190552 seconds ---
--- 8.916399955749512 seconds ---
--- 8.30895209312439 seconds ---
--- 9.189623832702637 seconds ---
--- 9.177680730819702 seconds ---
--- 8.421258926391602 seconds ---
--- 8.698073863983154 seconds ---
--- 9.262124061584473 seconds ---
--- 8.5989670753479 seconds ---
--- 8.357210874557495 seconds ---
--- 8.401618003845215 seconds ---
--- 9.036270141601562 seconds ---
--- 8.643677234649658 seconds ---
--- 9.415060997009277 seconds ---
--- 9.196885824203491 seconds ---
--- 8.574061155319214 seconds ---
--- 8.43940806388855 seconds ---
--- 9.223910808563232 seconds ---
--- 8.798089265823364 seconds ---
--- 7.941318035125732 seconds ---
--- 7.632917165756226 seconds ---
--- 5.106414079666138 seconds ---
--- 4.97188401222229 seconds ---
--- 4.7357041835784

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


--- 16.186964988708496 seconds ---
--- 9.561088800430298 seconds ---
--- 9.031822919845581 seconds ---
--- 8.496877908706665 seconds ---
--- 9.492054224014282 seconds ---
--- 9.240471124649048 seconds ---
--- 8.629431009292603 seconds ---
--- 8.975924015045166 seconds ---
--- 9.162899255752563 seconds ---
--- 8.33813214302063 seconds ---
--- 8.679069995880127 seconds ---
--- 9.42847490310669 seconds ---
--- 8.641788005828857 seconds ---
--- 8.478583097457886 seconds ---
--- 9.44046926498413 seconds ---
--- 8.991106033325195 seconds ---
--- 8.732371807098389 seconds ---
--- 8.132899045944214 seconds ---
--- 8.8486909866333 seconds ---
--- 8.643611192703247 seconds ---
--- 9.072847127914429 seconds ---
--- 8.915363073348999 seconds ---
--- 8.713746786117554 seconds ---
--- 8.2192702293396 seconds ---
--- 9.120890855789185 seconds ---
--- 8.69258999824524 seconds ---
--- 9.89629077911377 seconds ---
--- 7.7822699546813965 seconds ---
--- 5.11765193939209 seconds ---
--- 4.7950921058654785

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [7]:
# len(nature_file_list)+len(city_file_list)

In [ ]:
# for file_name in nature_file_list+city_file_list:
#     print(file_name)
#     run_preproc(file_name)